In [91]:
import json
import zipfile
import os
import sys
import io
from ast import literal_eval
import pandas as pd
from itertools import chain
import unicodedata
import numpy as np

from string import punctuation
translator = str.maketrans('','',punctuation) 

In [92]:
class repository:
    def __init__(self, policy, title, chapter, section, article):
        self.policy_ = policy
        self.title = title
        self.chapter = chapter
        self.section = section
        self.article = article
        
    @classmethod
    def from_repository_name(cls, rep_str):
        folder_parts = rep_str.split('_')
        policy = folder_parts[0] + '_' + folder_parts[1]
        if folder_parts[2] in  ['front', 'Whereas']:
            title = 'front'
            chapter = 'None'
            section = 'None'
            article = 'None'
        else:
            title = folder_parts[2] + '_' + folder_parts[3]
            chapter = folder_parts[4] + '_' + folder_parts[5]
            section = folder_parts[6] + '_' + folder_parts[7]
            article = folder_parts[8] + '_' + folder_parts[9]
        
        return cls(policy,title, chapter, section, article)
    
    def match(self, other):
        self_value_set = set([x for x in list(self.__dict__.values()) if x != 'None'])
        other_value_set = set([x for x in list(other.__dict__.values()) if x != 'None'])
        
        print(self_value_set)
        print(other_value_set)

        if set(other_value_set).issubset(self_value_set) == True:
            print('yes')
            
        else:
            print('no')
            
            
        print('-------------------')
        print('')
        return set(other_value_set).issubset(self_value_set)

    
class tag:
    def __init__(self, class_, type_, tag_, start, stop, rep):
        self.class_ = class_
        self.type_ = type_
        self.tag_ = tag_
        self.start = start
        self.stop = stop
        self.rep = rep
        
    def __eq__(self, other): 
        if not isinstance(other, tag):
            # don't attempt to compare against unrelated types
            return NotImplemented

        return self.class_ == other.class_ and self.type_ == other.type_ and self.tag_ == other.tag_ and self.start == other.start and self.stop == other.stop
    
    def __repr__(self): #for debugging purpose, defines how object is printed
        return "class:" + self.class_ + " type:" + self.type_ + " tag:" + self.tag_ + " start:" + str(self.start) + " stop:" + str(self.stop)
 
    
class Evaluator:
    
    def __init__(self, df):
        self.df = df
        
    def get_word_count(self, article):
        if article == 'None':
            return len(list(chain.from_iterable(self.df['tokens cleaned'])))
        
        if article in self.df['Policy'].tolist():
            return len(self.df.loc[self.df['Policy'] == article,'tokens cleaned'].iloc[0])
        
        else:
            raise Exception('This article is not present. Enter "None" to search all articles or a valid article')
    
        
        
    def _get_iterator_all(self): 
        
        return chain.from_iterable(self.df['Tags'])
        
        
        
    def _get_iterator_condition(self, conditional_rep): 
  
        iterator = chain.from_iterable(self.df['Tags'])
        return [x for x in iterator if type(x)!= str and x.rep.match(conditional_rep) == True]
    
      
        
        #else:
         #   raise Exception('This article is not present. Enter "None" to search all articles or a valid article')  
            
        
    
    def get_list(self, article, item, value):
        iterator = self._get_iterator(article)
        if item == 'class':
            return [x for x in iterator if type(x)!= str and x.class_ == value]
        if item == 'type':
            return [x for x in iterator if type(x)!= str and x.type_ == value]
        if item == 'tag':
            return [x for x in iterator if type(x)!= str and x.tag_ == value]
        
    def get_list_test(self, conditional_rep, item, value):
        #problem: how to get the matchings e.g only specifying the article but not the rest, meaning that none has to match to everything else
        #idea: do loop, when encountering a none then skip to next
        iterator = _get_iterator_condition(rep)
        if item == 'class':
            return [x for x in iterator if type(x)!= str and x.class_ == value]
        if item == 'type':
            return [x for x in iterator if type(x)!= str and x.type_ == value]
        if item == 'tag':
            return [x for x in iterator if type(x)!= str and x.tag_ == value]
            
     
    def get_frequency(self, article, item, value):
        return len(self.get_list(article, item, value))
      
  
          
        

In [93]:
def normalize_and_replace_text(sentence):
    doc = unicodedata.normalize("NFKD", sentence)
    doc = doc.replace('\r', '').replace('\n', ' ')

    
    return doc

def clean_text(doc):
    lower = doc.lower() # all lower case
    nopunc = lower.translate(translator) # remove punctuation
    words = nopunc.split() # split into tokens
    no_numbers = [w if not w.isdigit() else '#' for w in words]

    return no_numbers

def get_repository_names(folder_name):
    folder_parts = folder_name.split('_')
    policy = folder_parts[0] + '_' + folder_parts[1]
    title = folder_parts[2] + '_' + folder_parts[3]
    chapter = folder_parts[4] + '_' + folder_parts[5]
    section = folder_parts[6] + '_' + folder_parts[7]
    article = folder_parts[8] + '_' + folder_parts[9]
    
    
     
    return policy, title, chapter, section, article



In [94]:
stat_df = pd.DataFrame(columns = ['Policy', 'Text','Tags'])
tagsets = ['Policydesigncharacteristics','Technologyandapplicationspecificity','Instrumenttypes']


In [95]:
path = '/home/kueddelmaier/Downloads/full_project_curated_json/curation'
subdirs = [o for o in os.listdir(path) if os.path.isdir(path)]
os.chdir(path)

In [96]:
for subdir in subdirs:
    #since hete everyt token has same attributes, try to definde attributes policy and 
    try:
        archive = zipfile.ZipFile(os.path.join(subdir, str(os.listdir(subdir)[0])), 'r')
        json_file_byte = archive.read('CURATION_USER.json')
        
        json_file_byte_decode = json_file_byte.decode('utf8')
    
        data = json.loads(json_file_byte_decode)
    
    except:
        stat_df = stat_df.append(pd.Series([subdir, 'error', 'error'], index=stat_df.columns ), ignore_index=True)

        
    else:
        taglist = []
        rep = repository.from_repository_name(subdir[0:-4])
        sentence = data['_referenced_fss']['1']['sofaString']
        sentence_normalized = normalize_and_replace_text(sentence)
        
        for category in data['_views']['_InitialView']:
  
            if category in tagsets:
                    
                for annotation in data['_views']['_InitialView'][category]:
                               
                    taglist.append(tag(category, list(annotation.keys())[-1], list(annotation.values())[-1],annotation['begin'],annotation['end'], rep))
                    
        stat_df = stat_df.append(pd.Series([subdir[0:-4], sentence_normalized ,taglist], index=stat_df.columns ), ignore_index=True)     
stat_df['tokens cleaned'] = stat_df['Text'].apply(clean_text)    

In [97]:
stat_df.head('Pol1', 'title1', 'chapter1', 'section1', 'article1')

TypeError: head() takes from 1 to 2 positional arguments but 6 were given

In [98]:
stat_df['Tags'][28][8].rep.article

'Article_03'

In [99]:
hello = repository('None', 'Title_0', 'Chapter_0', 'None', 'None')
test_eval = Evaluator(stat_df)
full = test_eval._get_iterator_condition(hello)
full1 = test_eval._get_iterator_all()



{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32009

yes
-------------------

{'Title_0', 'Chapter_0', 'EU_32009L0028', 'Article_13', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'EU_32009L0028', 'Article_13', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'EU_32009L0028', 'Article_13', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'EU_32009L0028', 'Article_13', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'EU_32009L0028', 'Article_13', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'EU_32009L0028', 'Article_13', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'EU_32009L0028', 'Article_13', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'EU_32009L0028', 'Article_13', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0'

yes
-------------------

{'Title_0', 'Chapter_0', 'Article_06', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_06', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_06', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_06', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_06', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_06', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_06', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_06', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0'

{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_05', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_05', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_05', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_05', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_05', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_05', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_05', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_05', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0'}
no
-------------------

{'Title_0', 'EU_32018R1999', 'Article_02', 'Section_0', 'Chapter_1'}
{'Title_0', 'Chapter_0'}
no
-------------------

{'Title_0', 'EU_32018R1999', 'Article_02', 'Section_0', 'Chapter_1'}
{'Title_0', 'Chapter_0'}
no
-------------------

{'Title_0', 'EU_32018R1999', 'Article_02', 'Section_0', 'Chapter_1'}
{'Title_0', 'Chapter_0'}
no
-------------------

{'Title_0', 'EU_32018R1999', 'Article_02', 'Section_0', 'Chapter_1'}
{'Title_0', 'Chapter_0'}
no
-------------------

{'Title_0', 'EU_32018R1999', 'Article_02', 'Section_0', 'Chapter_1'}
{'Title_0', 'Chapter_0'}
no
-------------------

{'Title_0', 'EU_32018R1999', 'Article_02', 'Section_0', 'Chapter_1'}
{'Title_0', 'Chapter_0'}
no
-------------------

{'Title_0', 'EU_32018R1999', 'Article_02', 'Section_0', 'Chapter_1'}
{'Title_0', 'Chapter_0'}
no
-------------------

{'Title_0', 'EU_32018R1999', 'Article_02', 'Section_0', 'Chapter_1'}
{'Title_0', 'Chapter_0'}
no
-------------------

{'Title

{'Title_0', 'Chapter_0', 'Article_07', 'EU_32008R1099', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'Section_0', 'EU_32009R0079'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'Section_0', 'EU_32009R0079'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'Section_0', 'EU_32009R0079'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'Section_0', 'EU_32009R0079'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'Section_0', 'EU_32009R0079'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'Section_0', 'EU_32009R0079'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'Section_0', 'EU_32009R0079'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'Section_

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009L0028', 'Section_0'}
{'Title_0', 'Chapter_0'}
yes
-------------------

{'Title_0', 'Chapter_0', 'Article_15', 'EU_32009

In [82]:
len(list(full))


3648

In [59]:
search_dir = repository('None', 'None', 'None', 'section1', 'article1')
testdir_else = repository('None', 'None', 'chapter1', 'section1', 'article1')
search_dir_list = list(search_dir.__dict__.values())
testdir_else_list = list(testdir_else.__dict__.values())
print(search_dir_list)
print(testdir_else_list)
#print([x for x in search_dir_list if x in testdir_else_list and x  != 'None'])
set([x for x in search_dir_list if x != 'None']).issubset(set(testdir_else_list)) 
        
        

['None', 'None', 'None', 'section1', 'article1']
['None', 'None', 'chapter1', 'section1', 'article1']


True

In [ ]:

final = {dic2[k]: v for k, v in dic1.items()}

In [35]:
list(test_dir.__dict__.values())

['Pol1', 'title1', 'chapter1', 'section1', 'article1']

In [21]:
stat_df.loc[stat_df['Policy'] == 'EU_32019R0631_Title_0_Chapter_0_Section_0_Article_16.','Tags'].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [22]:
stat_df[]

SyntaxError: invalid syntax (<ipython-input-22-e45f6d74c8bf>, line 1)

In [94]:
subset = stat_df['Tags'][0:2]
for set_ in subset:
    for entry in set_:
        print(entry.class_)




Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity
Technologyandapplicationspecificity


In [97]:
[x for b in subset for x in b if x.class_ == 'Technologyandapplicationspecificity'] 

In [110]:
stat_df['']

NameError: name '__main__' is not defined

In [51]:
len([x for x in chain.from_iterable(stat_df['Tags']) if type(x)!= str and x.class_ == 'Technologyandapplicationspecificity']) 

1344

In [32]:
data['_views']['_InitialView']['Token']

[{'sofa': 1, 'end': 7},
 {'sofa': 1, 'begin': 7, 'end': 8},
 {'sofa': 1, 'begin': 8, 'end': 10},
 {'sofa': 1, 'begin': 12, 'end': 21},
 {'sofa': 1, 'begin': 22, 'end': 24},
 {'sofa': 1, 'begin': 25, 'end': 28},
 {'sofa': 1, 'begin': 29, 'end': 35},
 {'sofa': 1, 'begin': 36, 'end': 42},
 {'sofa': 1, 'begin': 44, 'end': 45},
 {'sofa': 1, 'begin': 45, 'end': 46},
 {'sofa': 1, 'begin': 46, 'end': 49},
 {'sofa': 1, 'begin': 49, 'end': 53},
 {'sofa': 1, 'begin': 54, 'end': 60},
 {'sofa': 1, 'begin': 61, 'end': 66},
 {'sofa': 1, 'begin': 67, 'end': 72},
 {'sofa': 1, 'begin': 73, 'end': 79},
 {'sofa': 1, 'begin': 80, 'end': 81},
 {'sofa': 1, 'begin': 82, 'end': 88},
 {'sofa': 1, 'begin': 89, 'end': 91},
 {'sofa': 1, 'begin': 92, 'end': 95},
 {'sofa': 1, 'begin': 96, 'end': 106},
 {'sofa': 1, 'begin': 107, 'end': 109},
 {'sofa': 1, 'begin': 110, 'end': 118},
 {'sofa': 1, 'begin': 119, 'end': 121},
 {'sofa': 1, 'begin': 122, 'end': 125},
 {'sofa': 1, 'begin': 126, 'end': 135},
 {'sofa': 1, 'begi

In [72]:
list(data['_views']['_InitialView']['Technologyandapplicationspecificity'][0].values())

[1, 57, 88, 'Tech_LowCarbon']

In [74]:
test_df = pd.DataFrame(columns = ['text'], index = np.arange(0,2))
test_df['text'].loc[0] = 'hello peter ist mee'
test_df['text'].loc[1] = 'why I am su sotun'

In [75]:
test_df.head()

,text
0,hello peter ist mee
1,why I am su sotun


In [81]:
test_df['text'] = test_df['text'].apply(clean_text)

In [85]:

len(list(chain.from_iterable(test_df['text'])))

9

In [38]:
class Negator(object):
    def __eq__(self,other):
        return not other

thing = Negator()
print(thing == None)    #True
print(thing is None)    #False

True
False
